In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta
from itertools import combinations
import seaborn as sns
import os, sys, warnings
from time import time 
from create_feature import *
from glob import glob
import lightgbm as lgb
import joblib, gc
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sys 
# add path of optiver2023 package to pythonpath 
sys.path.append(os.path.abspath('../data'))
from create_feature import * 

import optiver2023 

In [2]:
train_par = pd.read_parquet("/home/lishi/projects/Competition/kaggle_2023/data/train_add_daily_features.parquet")

feature_cols = [x for x in train_par.columns if x not in ['row_id', 'target', 'time_id']]

In [3]:
df = pd.read_csv("/home/lishi/projects/Competition/kaggle_2023/data/train.csv")
df = df[~df['target'].isnull()].reset_index(drop=True)

In [4]:
prices =  ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
category = ["stock_id", "seconds_in_bucket", 'imbalance_buy_sell_flag', 'stock_label']

stock_labels = pd.read_csv("/home/lishi/projects/Competition/kaggle_2023/stock_labels.csv")
stock_labels.columns = ['stock_id', 'stock_label']

n_days = 5 # rolling window on date_id
n_seconds = 5 # rolling window on seconds_in_bucket

In [5]:
# construct initial df_records and df_revealed
last_train_day = df['date_id'].max()

print("The last train day is {}".format(last_train_day))

df_trian = df[df['date_id'] >= last_train_day - n_days].reset_index(drop=True)

df_records = df_trian[df_trian.columns.difference(['target', 'row_id', 'time_id'])].copy()
df_records['currently_scored'] = False

df_records, v1_features, v2_features = gen_v1v2_features(df_records, prices, sizes, stock_labels)

df_revealed = df_trian[['date_id', 'stock_id', 'seconds_in_bucket', 'target']].copy()

The last train day is 480


In [6]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [7]:
counter = 0 
n_reveals = 0

day_begin = time()

for (test, revealed_targets, sample_prediction) in iter_test:
    
    now_time = time()
    
    curr_date = test['date_id'].unique().tolist()[0]
    curr_seconds = test['seconds_in_bucket'].unique().tolist()[0]
    test = test.drop(['row_id'], axis=1)
    test_cols = test.columns.tolist()
    
    print("curr_date", curr_date)
    
    if counter == 0:
        n_reveals += 1
        print('Targets revealed for ', revealed_targets['revealed_date_id'].unique().tolist())
        
        if n_reveals > 1:
            tmp = revealed_targets[['revealed_date_id', 'stock_id', 'seconds_in_bucket', 'revealed_target']]
            tmp.columns = ['date_id', 'stock_id', 'seconds_in_bucket', 'target']
            df_revealed = pd.concat([df_revealed, tmp])
            
        if df_revealed['date_id'].nunique() > n_days+1:
            df_revealed = df_revealed[df_revealed['date_id'] != df_revealed['date_id'].min()]

        # calculate targets for past days 
        xy_merge = pd.merge(
            df_records,
            df_revealed, 
            on=['date_id', 'stock_id', 'seconds_in_bucket'], 
            how='left')
        
        daily_stats, daily_features = gen_daily_stats_features(xy_merge, n_days=n_days)
        
        # gp = xy_merge.groupby(['stock_id', 'seconds_in_bucket'])
        # on_cols=['target', 'wap', 'volume']
        
        # for i in range(1, n_days+1):
        #     daily_shift = pd.concat(
        #         [df]+ [gp[on_cols].shift(i).add_prefix(f'prev_{i}d_') for i in range(1, n_days+1)], axis=1)
            
        # daily_features = []
        # stats_list = []
        
        # for col in on_cols:
        #     prev_cols = [f'prev_{i}d_{col}' for i in range(1, n_days+1)]
            
        #     df_stats = pd.concat([df[prev_cols].mean(axis=1), df[prev_cols].std(axis=1), df[prev_cols].max(axis=1), df[prev_cols].min(axis=1)], axis=1)
        #     stats_cols = [f'mean_{col}_{n_days}d', f'std_{col}_{n_days}d', f'max_{col}_{n_days}d', f'min_{col}_{n_days}d']
            
        #     df_stats.columns = stats_cols
            
        #     daily_features += stats_cols
        #     stats_list.append(df_stats)
            
        #     df = df.drop(columns=prev_cols)
        
        # df = pd.concat([df] + stats_list, axis=1)
    
    test, v1_features, v2_features = gen_v1v2_features(test, prices, sizes, stock_labels)
    
    df_records = pd.concat([df_records, test], axis=0)
    
    cache = df_records[
        (df_records['date_id'] == curr_date) & 
        (df_records['seconds_in_bucket'] <= curr_seconds) &
        (df_records['seconds_in_bucket'] >= curr_seconds-n_seconds*10)
        ].copy()
    
    cache, v3_features = gen_v3_features(cache)
    
    cache = cache[cache['seconds_in_bucket'] == curr_seconds]
    
    merge_key = ['date_id', 'stock_id', 'seconds_in_bucket']
    
    cache = pd.merge(cache, daily_stats[merge_key+daily_features], on=merge_key, how='left')
    
    x_train = cache[ (cache['date_id']==curr_date) & (cache['seconds_in_bucket'] == curr_seconds) ]
    
    
    sample_prediction['target'] = 0
    env.predict(sample_prediction)

    # after 54 timesteps, a new day starts
    if counter >= 54:
        print(f"New Day! Time used: {time() - day_begin:2f}s.")
        counter = 0
        day_begin = time()
        cache = pd.DataFrame()
        
        # kick out the oldest date
        df_records = df_records[df_records['date_id'] != df_records['date_id'].min()]
    else:
        counter += 1
        
        
    

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
curr_date 478
Targets revealed for  [477]
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
curr_date 478
New Day! Time used: 12.759804s.
curr_date 479
Targets revealed for  [478]
cur

In [8]:
df_records

,ask_price,ask_size,bid_price,bid_size,date_id,far_price,imbalance_buy_sell_flag,imbalance_size,matched_size,near_price,...,rank_depth_pressure,rank_price_pressure,rank_imbalance_with_flag,rank_far_price_near_price_imbalance,rank_reference_price_ask_price_imbalance,rank_reference_price_bid_price_imbalance,rank_reference_price_wap_imbalance,rank_ask_price_bid_price_imbalance,rank_ask_price_wap_imbalance,rank_bid_price_wap_imbalance
33000,1.000050,9177.599609,0.999875,22940.000000,478,0.000000,-1,3.753452e+06,11548975.0,0.000000,...,0.0000,0.4400,0.1450,0.0000,0.610,0.2775,0.3450,0.070,0.085,0.6475
33001,1.000601,19692.000000,0.999940,1967.900024,478,0.000000,-1,9.859771e+05,3850034.0,0.000000,...,0.0000,0.4250,0.3250,0.0000,0.465,0.6000,0.7500,0.600,0.745,0.8100
33002,1.000636,34955.121094,0.999918,4488.220215,478,0.000000,1,5.991288e+05,4359198.0,0.000000,...,0.0000,0.3300,0.6500,0.0000,0.685,0.8400,0.9000,0.615,0.775,0.7375
33003,1.000189,10314.000000,0.999705,16082.040039,478,0.000000,-1,2.872318e+06,27129552.0,0.000000,...,0.0000,0.6150,0.1850,0.0000,0.425,0.4150,0.3125,0.405,0.355,0.3350
33004,1.000107,7245.600098,0.999720,19012.349609,478,0.000000,-1,7.400591e+05,8880891.0,0.000000,...,0.0000,0.2600,0.3600,0.0000,0.575,0.4750,0.3625,0.335,0.170,0.3650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.000434,319862.406250,1.000317,32257.039062,480,0.999734,-1,2.440723e+06,28280362.0,0.999734,...,0.5675,0.6650,0.1300,0.5225,0.440,0.2100,0.4500,0.215,0.465,0.9150
196,1.000900,93393.070312,1.000643,205108.406250,480,1.000129,-1,3.495105e+05,9187699.0,1.000386,...,0.9050,0.4300,0.3350,0.1850,0.200,0.2100,0.0800,0.600,0.335,0.2300
197,0.995883,180038.312500,0.995789,16790.660156,480,0.995789,0,0.000000e+00,12725436.0,0.995789,...,0.5675,0.1225,0.5375,0.5225,0.495,0.2100,0.4650,0.140,0.385,0.9400
198,0.999210,669893.000000,0.998970,125631.718750,480,0.999210,1,1.000899e+06,94773272.0,0.999210,...,0.5675,0.6150,0.8300,0.5225,0.835,0.7900,0.8850,0.570,0.685,0.7200


In [14]:
# for i in range(max(len(x_train.columns), len(feature_cols))):
#     if x_train.columns[i] != feature_cols[i]:
#         print(x_train.columns[i], feature_cols[i])

for x in feature_cols:
    if x not in x_train.columns:
        print(x)

ema
rsi
cci
mfi
ad_osc
macd
macdsignal
macdhist


In [11]:
len(feature_cols)

344